In [1]:
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import os.path
import IPython.display
import seaborn as sns
import librosa
import librosa.display
import soundfile
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers

from keras import optimizers,regularizers
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.decomposition import PCA
import warnings

In [2]:
!pip install pydub
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import re
import os
import wave
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from IPython.display import Audio, display
import librosa as lib
import librosa.display

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
PATH = "/content/drive/MyDrive/Colab Notebooks/Crema"
AUDIO_PATH = "/content/drive/MyDrive/Colab Notebooks/Crema/1001_DFA_ANG_XX.wav"

In [6]:
def zeroCrossingRate(audio):
  return lib.feature.zero_crossing_rate(audio)

In [7]:
def energy(audio):
  # using a spectrogram will give a more accurate representation
  # of energy over time because its frames can be windowed
  S, phase = lib.magphase(lib.stft(audio))
  return lib.feature.rms(S=S).mean()

In [8]:
def chromaStft(audio, sr):
  stft = np.abs(librosa.stft(audio))
  return librosa.feature.chroma_stft(S=stft, sr=sr, n_fft=200).mean()

In [9]:
def mfcc(data, sr):
  return librosa.feature.mfcc(y=data, sr=sr, n_fft=200).mean()

In [10]:
def melSpectrogram(audio, sr):
  mel_spectrogram = lib.feature.melspectrogram(y=audio, sr=sr, n_fft=200, n_mels=64)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram

In [11]:
def playAudio(audio_file):
  audio = AudioSegment.from_wav(audio_file)
  # Play the audio
  audio.export('temp_audio.wav', format='wav')
  audio_data = open('temp_audio.wav', 'rb').read()
  display(Audio(audio_data))
  # Delete the temporary audio file
  os.remove('temp_audio.wav')

In [12]:
def visualize_waveform(audio, sr):
    plt.figure(figsize=(12, 4))
    plt.plot(audio)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.show()

In [13]:
def splitData(dataset_path):
  audio_files = glob.glob(os.path.join(dataset_path, "*.wav"))
  audio_train, audio_test = train_test_split(audio_files, test_size=0.3, random_state=42, shuffle=True)
  return audio_train, audio_test

In [14]:
D_train, D_test = splitData(PATH)

In [15]:
print(len(D_train), len(D_test))

5216 2236


In [16]:
D_test[0]

'/content/drive/MyDrive/Colab Notebooks/Crema/1017_IEO_SAD_HI.wav'

In [17]:
def getMaxLen(audio_files):
  max_length = 0
  for audio_file in audio_files:
    audio, _ = librosa.load(audio_file)
    length = len(audio)
    max_length = max(max_length, length)
  return max_length

In [18]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  # for data augmentation
  augmentations = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
])
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc, mel_spec], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    # don't augment every example
    if np.random.rand() >= 0.5:
      augmented_audio = augmentations(samples=audio, sample_rate=sr)
      # extract zero crossing rate
      zcr = zeroCrossingRate(augmented_audio)
      # extract energy
      rms = energy(augmented_audio)
      # chroma stft
      cs = chromaStft(augmented_audio, sr)
      # mfcc
      mfc = mfcc(augmented_audio, sr)
      # extract mel spectrogram
      mel_spec = melSpectrogram(augmented_audio, sr).mean()
      combined_features = np.concatenate(([rms, cs, mfc, mel_spec], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
      D.append(combined_features)
      for cls in classes:
        if re.search(cls, audio_file): Y.append(cls)
  return D, Y

In [19]:
def loadAudioFreq(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  # for data augmentation
  augmentations = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
])
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    D.append(mel_spec)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    # don't augment every example
    if np.random.rand() >= 0.5:
      augmented_audio = augmentations(samples=audio, sample_rate=sr)
      mel_spec_aug = melSpectrogram(augmented_audio, sr)
      D.append(mel_spec_aug)
      for cls in classes:
        if re.search(cls, audio_file): Y.append(cls)
  return D, Y

## Time Domain Model

In [20]:
D_time, Y_time = loadAudioTime(D_train)

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


In [21]:
D_time = np.array(D_time)
D_time.shape

(7808, 304)

In [22]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [23]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [24]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(7808, 6)


In [25]:
Y_time = one_hot_labels
Y_time.shape

(7808, 6)

In [26]:
# Apply PCA to the data
pca = PCA(n_components=200)
D_pca = pca.fit_transform(D_time)

In [27]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((7417, 200), (7417, 6), (391, 200), (391, 6))
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 198, 128)          512       
                                                                 
 batch_normalization (BatchN  (None, 198, 128)         512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 198, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 99, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 97, 256)           98560     
                                                                 
 batch_no

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
232/232 [==============================] - 16s 24ms/step - loss: 3.2911 - accuracy: 0.2916 - val_loss: 3.3941 - val_accuracy: 0.1480
Epoch 2/50
232/232 [==============================] - 5s 21ms/step - loss: 2.6746 - accuracy: 0.3311 - val_loss: 2.8992 - val_accuracy: 0.1545
Epoch 3/50
232/232 [==============================] - 4s 19ms/step - loss: 2.4486 - accuracy: 0.3457 - val_loss: 2.4969 - val_accuracy: 0.3114
Epoch 4/50
232/232 [==============================] - 4s 18ms/step - loss: 2.3190 - accuracy: 0.3545 - val_loss: 2.2743 - val_accuracy: 0.3384
Epoch 5/50
232/232 [==============================] - 5s 21ms/step - loss: 2.2200 - accuracy: 0.3628 - val_loss: 2.1614 - val_accuracy: 0.3609
Epoch 6/50
232/232 [==============================] - 4s 18ms/step - loss: 2.1300 - accuracy: 0.3783 - val_loss: 2.2591 - val_accuracy: 0.3166
Epoch 7/50
232/232 [==============================] - 4s 19ms/step - loss: 2.0588 - accuracy: 0.3843 - val_loss: 2.0014 - val_accuracy: 0.404

## Frequency Domain Model

In [28]:
D_freq, Y_freq = loadAudioFreq(D_train)

In [29]:
unicheck = np.unique(Y_freq, axis=0)
print(f"Check on freq domain:\n{unicheck}")

Check on freq domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [30]:
df_freq = pd.DataFrame(Y_freq)
df_freq.head()

,0
0,SAD
1,SAD
2,FEA
3,FEA
4,SAD


In [31]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_freq)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(7783, 6)


In [32]:
Y_freq = one_hot_labels
Y_freq.shape

(7783, 6)

In [33]:
for i, array in enumerate(D_freq):
    D_freq[i] = np.pad(array, ((0, 0), (0, 256 - array.shape[1])))
D_freq = np.array(D_freq)

In [34]:
D_freq.shape

(7783, 64, 256)

In [35]:
x_train, x_test, y_train, y_test = train_test_split(np.array(D_freq), np.array(Y_freq), test_size=0.05)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((7393, 64, 256), (7393, 6), (390, 64, 256), (390, 6))


In [36]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
print(x_train.shape, x_test.shape)

(7393, 64, 256, 1) (390, 64, 256, 1)


In [37]:
model = Sequential()

# Layer 1: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Flatten Layer
model.add(Flatten())

# Layer 5: Fully Connected Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Layer 6: Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Layer 7: Output Layer
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 254, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 127, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 125, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 30, 128)      

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [38]:
# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Epoch 1/50
58/58 [==============================] - 7s 86ms/step - loss: 1.8537 - accuracy: 0.2262 - val_loss: 1.6629 - val_accuracy: 0.3077
Epoch 2/50
58/58 [==============================] - 3s 51ms/step - loss: 1.6407 - accuracy: 0.3099 - val_loss: 1.5287 - val_accuracy: 0.3513
Epoch 3/50
58/58 [==============================] - 3s 50ms/step - loss: 1.5627 - accuracy: 0.3507 - val_loss: 1.5019 - val_accuracy: 0.3308
Epoch 4/50
58/58 [==============================] - 3s 52ms/step - loss: 1.5377 - accuracy: 0.3528 - val_loss: 1.4941 - val_accuracy: 0.3846
Epoch 5/50
58/58 [==============================] - 3s 51ms/step - loss: 1.5065 - accuracy: 0.3764 - val_loss: 1.4397 - val_accuracy: 0.3795
Epoch 6/50
58/58 [==============================] - 3s 53ms/step - loss: 1.4941 - accuracy: 0.3827 - val_loss: 1.4399 - val_accuracy: 0.3872
Epoch 7/50
58/58 [==============================] - 3s 50ms/step - loss: 1.4731 - accuracy: 0.3983 - val_loss: 1.4213 - val_accuracy: 0.3974
Epoch 8/50
58

In [39]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  # for data augmentation
  augmentations = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
])
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    #mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    # don't augment every example
    if np.random.rand() >= 0.5:
      augmented_audio = augmentations(samples=audio, sample_rate=sr)
      # extract zero crossing rate
      zcr = zeroCrossingRate(augmented_audio)
      # extract energy
      rms = energy(augmented_audio)
      # chroma stft
      cs = chromaStft(augmented_audio, sr)
      # mfcc
      mfc = mfcc(augmented_audio, sr)
      # extract mel spectrogram
      #mel_spec = melSpectrogram(augmented_audio, sr).mean()
      combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
      D.append(combined_features)
      for cls in classes:
        if re.search(cls, audio_file): Y.append(cls)
  return D, Y

In [40]:
D_time, Y_time = loadAudioTime(D_train)

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


In [41]:
D_time = np.array(D_time)
D_time.shape

(7835, 303)

In [42]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [43]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,SAD
4,ANG


In [44]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(7835, 6)


In [45]:
Y_time = one_hot_labels
Y_time.shape

(7835, 6)

In [46]:
# Apply PCA to the data
pca = PCA(n_components=200)
D_pca = pca.fit_transform(D_time)

In [47]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((7443, 200), (7443, 6), (392, 200), (392, 6))
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 198, 128)          512       
                                                                 
 batch_normalization_7 (Batc  (None, 198, 128)         512       
 hNormalization)                                                 
                                                                 
 activation_7 (Activation)   (None, 198, 128)          0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 99, 128)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 97, 256)           98560     
                                                                 
 batch_

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
233/233 [==============================] - 8s 23ms/step - loss: 3.4641 - accuracy: 0.2769 - val_loss: 3.5970 - val_accuracy: 0.1464
Epoch 2/50
233/233 [==============================] - 4s 18ms/step - loss: 2.8334 - accuracy: 0.3184 - val_loss: 3.0989 - val_accuracy: 0.1493
Epoch 3/50
233/233 [==============================] - 5s 22ms/step - loss: 2.5740 - accuracy: 0.3364 - val_loss: 2.4826 - val_accuracy: 0.3419
Epoch 4/50
233/233 [==============================] - 5s 19ms/step - loss: 2.4003 - accuracy: 0.3614 - val_loss: 2.3931 - val_accuracy: 0.3345
Epoch 5/50
233/233 [==============================] - 4s 18ms/step - loss: 2.2647 - accuracy: 0.3800 - val_loss: 2.5308 - val_accuracy: 0.2851
Epoch 6/50
233/233 [==============================] - 5s 21ms/step - loss: 2.1824 - accuracy: 0.3740 - val_loss: 2.3478 - val_accuracy: 0.2699
Epoch 7/50
233/233 [==============================] - 4s 18ms/step - loss: 2.0951 - accuracy: 0.3910 - val_loss: 2.1000 - val_accuracy: 0.3787

**Without Augmentation**

In [48]:
def getMaxLen(audio_files):
  max_length = 0
  for audio_file in audio_files:
    audio, _ = librosa.load(audio_file)
    length = len(audio)
    max_length = max(max_length, length)
  return max_length

In [75]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  #1D WITHOUT MEL
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    #mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
  return D,Y
  

In [55]:
def loadAudioFreq(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    D.append(mel_spec)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    
  return D, Y

In [56]:
D_time, Y_time = loadAudioTime(D_train)

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


In [57]:
D_time = np.array(D_time)
D_time.shape

(5216, 303)

In [58]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [59]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [60]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [61]:
Y_time = one_hot_labels
Y_time.shape

(5216, 6)

In [62]:
# Apply PCA to the data
pca = PCA(n_components=200)
D_pca = pca.fit_transform(D_time)

In [63]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((4955, 200), (4955, 6), (261, 200), (261, 6))
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 198, 128)          512       
                                                                 
 batch_normalization_14 (Bat  (None, 198, 128)         512       
 chNormalization)                                                
                                                                 
 activation_14 (Activation)  (None, 198, 128)          0         
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 99, 128)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 97, 256)           98560     
                                                                 
 batch_

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
155/155 [==============================] - 8s 32ms/step - loss: 3.5517 - accuracy: 0.3017 - val_loss: 3.8160 - val_accuracy: 0.1487
Epoch 2/50
155/155 [==============================] - 4s 27ms/step - loss: 2.9751 - accuracy: 0.3453 - val_loss: 3.9389 - val_accuracy: 0.1485
Epoch 3/50
155/155 [==============================] - 4s 23ms/step - loss: 2.7073 - accuracy: 0.3556 - val_loss: 3.1230 - val_accuracy: 0.1713
Epoch 4/50
155/155 [==============================] - 4s 24ms/step - loss: 2.5311 - accuracy: 0.3713 - val_loss: 2.7442 - val_accuracy: 0.2367
Epoch 5/50
155/155 [==============================] - 4s 26ms/step - loss: 2.4056 - accuracy: 0.3816 - val_loss: 2.6156 - val_accuracy: 0.2767
Epoch 6/50
155/155 [==============================] - 3s 22ms/step - loss: 2.3046 - accuracy: 0.4014 - val_loss: 2.2470 - val_accuracy: 0.4113
Epoch 7/50
155/155 [==============================] - 3s 22ms/step - loss: 2.2187 - accuracy: 0.4061 - val_loss: 2.4193 - val_accuracy: 0.3407

In [64]:
D_freq, Y_freq = loadAudioFreq(D_train)

In [65]:
unicheck = np.unique(Y_freq, axis=0)
print(f"Check on freq domain:\n{unicheck}")

Check on freq domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [66]:
df_freq = pd.DataFrame(Y_freq)
df_freq.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [67]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_freq)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [68]:
Y_freq = one_hot_labels
Y_freq.shape

(5216, 6)

In [69]:
for i, array in enumerate(D_freq):
    D_freq[i] = np.pad(array, ((0, 0), (0, 256 - array.shape[1])))
D_freq = np.array(D_freq)

In [70]:
D_freq.shape

(5216, 64, 256)

In [71]:
x_train, x_test, y_train, y_test = train_test_split(np.array(D_freq), np.array(Y_freq), test_size=0.05)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((4955, 64, 256), (4955, 6), (261, 64, 256), (261, 6))


In [72]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
print(x_train.shape, x_test.shape)

(4955, 64, 256, 1) (261, 64, 256, 1)


In [73]:
model = Sequential()

# Layer 1: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Flatten Layer
model.add(Flatten())

# Layer 5: Fully Connected Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Layer 6: Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Layer 7: Output Layer
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 254, 32)       320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 127, 32)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 125, 64)       18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 60, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 30, 128)      

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [74]:
# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Epoch 1/50
39/39 [==============================] - 4s 90ms/step - loss: 1.9025 - accuracy: 0.2194 - val_loss: 1.5908 - val_accuracy: 0.4061
Epoch 2/50
39/39 [==============================] - 2s 50ms/step - loss: 1.6561 - accuracy: 0.3021 - val_loss: 1.5124 - val_accuracy: 0.4253
Epoch 3/50
39/39 [==============================] - 2s 53ms/step - loss: 1.5916 - accuracy: 0.3330 - val_loss: 1.4681 - val_accuracy: 0.3946
Epoch 4/50
39/39 [==============================] - 2s 53ms/step - loss: 1.5523 - accuracy: 0.3514 - val_loss: 1.4942 - val_accuracy: 0.3640
Epoch 5/50
39/39 [==============================] - 2s 51ms/step - loss: 1.5189 - accuracy: 0.3756 - val_loss: 1.4540 - val_accuracy: 0.4061
Epoch 6/50
39/39 [==============================] - 2s 51ms/step - loss: 1.4986 - accuracy: 0.3796 - val_loss: 1.4173 - val_accuracy: 0.4406
Epoch 7/50
39/39 [==============================] - 2s 51ms/step - loss: 1.4694 - accuracy: 0.3964 - val_loss: 1.4275 - val_accuracy: 0.4330
Epoch 8/50
39